# CPSC 4300/6300-001 Applied Data Science (Fall 2020)

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Joshua Lin"
COLLABORATORS = ""

# CPSC4300/6300-001 Problem Set #1

In this assignment, you will create a NCAA College Football team dataset by scraping data from the Internet and perform some basic data manipulations. The assignment has four parts: the first three are auto-graded. Th elast part is manually graded by the TA. 

Before submission, make sure you run all the cells. Because the TA could not run your notebook, your grade will be only based on the output and the writing part of your submitted notebook. 

## Part B: Scape the Rosters of Any NCAA Collge Football Team (50 points)

During the class, we have shown how to extract the roster of the Clemson Tiger's defense team, now we want to extend the work to apply to the entire team roster of any NCAA coolege football teams.

### Question 1. Wriet a function to computer the Team URL

Look at the URLs of server college football team at the ESPN site, generalize the pattern, and write a python function  `get_team_url(team_id, team_name)` to compute a url from team_id and team_name. (4 points)

In [2]:
def get_team_url(team_id, team_name):
    """
    Get the team URL for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        team_url: a string representing the URL of the team
    """
    # YOUR CODE HERE
    #raise NotImplementedError()
    url = "https://www.espn.com/college-football/team/roster/_/id/" + str(team_id) + "/"
    real_team_name = team_name.split( )
    #print(real_team_name)
    i = 1
    while (len(real_team_name) > i):     
        url = url + real_team_name[i-1].lower()+"-"
        i = i + 1
    else: 
        url = url + real_team_name[i-1].lower()
    #print(url)
    return url

In [3]:
team_id = 228
team_name = "Clemson Tigers"
team_url = get_team_url(team_id, team_name)
assert team_url == 'https://www.espn.com/college-football/team/roster/_/id/228/clemson-tigers'

In [4]:
# This cell contains some hidden test cases.

### Question 2. Create a BeautifulSoup Object

Complete the following code to create a BeautifulSoup Object from a team url and name the object `soup`. (4 points)

In [5]:
import requests
import re
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError

url = get_team_url(228, "Clemson Tigers")

# YOUR CODE HERE
#raise NotImplementedError()
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, 'html.parser')
#soup

In [6]:
assert (type(soup) == BeautifulSoup)

In [7]:
assert soup.find('title').text.startswith('Clemson')

### Question 3. Find the Team Name

Write a statement to find a tag whose text is formatted as "{Team Name} Rosters" on the page. The {Team Name} would be the actual team name like 'Clemson Tigers'. (2 points)

In [8]:
# YOUR CODE HERE
#raise NotImplementedError()
team_tag = soup.find('h1', string=re.compile(' Roster$'))

In [9]:
assert str(type(team_tag)) == "<class 'bs4.element.Tag'>" and team_tag.text.endswith("Roster")

### Question 5. Search for Team Groups

A football team roster is usually divied into three groups: *Offense*, *Defense*, and *Special Teams*. Write some statements to find the tags for all these three groups and save the tags into a variable called groups. (2 points)

In [10]:
# YOUR CODE HERE
#raise NotImplementedError()
groups = soup.find_all(class_="Table__Title", string=["Offense", "Defense", "Special Teams"])
print(len(groups), "groups")
for g in groups:
    print(g)

3 groups
<div class="Table__Title">Offense</div>
<div class="Table__Title">Defense</div>
<div class="Table__Title">Special Teams</div>


In [11]:
assert ','.join(sorted([g.text for g in groups])) == 'Defense,Offense,Special Teams'

In [12]:
assert len(groups) == 3

### Question 6. Find the Column Names

Complete the code segment to extract the names of the columns in a table rows and save those names into a list named col_names. (4 points)

In [13]:
col_names = []
for g in groups:
    t = g.find_next("table")
    for c in t.thead.tr.children:
        col_names.append(c.text)
# YOUR CODE HERE
#raise NotImplementedError()
#col_names

In [14]:
assert "Name" in col_names and "Class" in col_names

In [15]:
assert type(col_names) is list and len(col_names) >= 6

### Question 7. Find the Attributes of a Player

Write code segment to extract the attributes of the first defense player and save them into a dict. The player should have the attributes as shown in the following example.
```
player_name: Lannden Zanders
player_id: 4428971
player_no: 36
POS: S
HT: 6' 1"
WT: 195 lbs
Class: SO
Birthplace: Shelby, NC
```
(10 points)

In [16]:
player = {}
players = []
# YOUR CODE HERE
#raise NotImplementedError()
table = groups[1].find_next("table")
def is_player_row(tag):
    if tag.name == 'tr' and tag.has_attr('data-idx'):
        return True
    return False
for c in table.tbody.find_all(is_player_row):
    tds = c.find_all('td')
    #print(tds)
    cols = []
    for td in tds:
        cols.append(td.get_text())
    players.append(cols)
    
def mysplit(s, ex):
    head = s.rstrip(ex)
    tail = s[len(head):]
    return head, tail

name = mysplit(players[-1][1], '0123456789')
player["player_name"] = name[0]

player_url_name = name[0].split()
totalURL = ""
i = 1
for x in player_url_name :
    if i != len(player_url_name) :
        if x.find(".") :
            x = x.replace('.', '') 
        totalURL = totalURL + x.lower() + "-"
    else :
        if x.find(".") :
            x = x.replace('.', '')
        totalURL = totalURL + x.lower()
    i = i + 1
totalURL = '/' + totalURL

id = soup.find('a', href=re.compile(totalURL))['href']
id = re.sub('http://www.espn.com/college-football/player/_/id/', '', id)
id = re.sub(totalURL, '', id)

player["player_id"] = id
player["player_no"] = name[1]
player["POS"] = players[-1][2]
player["HT"] = players[-1][3]
player["WT"] = players[-1][4]
player["Class"] = players[-1][5]
player["Birthplace"] = players[-1][6]
print(player)

{'player_name': 'Lannden Zanders', 'player_id': '4428971', 'player_no': '36', 'POS': 'S', 'HT': '6\' 1"', 'WT': '200 lbs', 'Class': 'SO', 'Birthplace': 'Shelby, NC'}


In [17]:
assert type(player) is dict

In [18]:
assert len(player) == 8

In [19]:
assert ','.join(sorted(player.keys())) == 'Birthplace,Class,HT,POS,WT,player_id,player_name,player_no'

In [20]:
assert int(player['player_no']) > 0 and int(player['player_no']) < 200

In [21]:
assert int(player['WT'].split()[0]) > 150 and player['WT'].split()[1] == 'lbs'

### Question 8. Abstract the tasks into a function

Since you don't want to repeat the above steps for every player and every team, it is necessary to abstract the above scraping task into a function which you can reuse over and over. (10 points)

In [22]:
import requests
import re
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup

def extract_team_roster(team_id, team_name):
    """
    Extract the team roster for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        a list of dict representing the roster of the entire team.
    """
    print("wrong")
    #team_id = 150
    #team_name = "Duke Blue Devils"
    roster = []
    def mysplit(s, ex):
        head = s.rstrip(ex)
        tail = s[len(head):]
        return head, tail
    
    def is_player_row(tag):
        if tag.name == 'tr' and tag.has_attr('data-idx'):
            return True
        return False
    
    #Computing Team URL
    url = "https://www.espn.com/college-football/team/roster/_/id/" + str(team_id) + "/"
    real_team_name = team_name.split( )
    #print(real_team_name)
    i = 1
    while (len(real_team_name) > i):     
        url = url + real_team_name[i-1].lower()+"-"
        i = i + 1
    else: 
        url = url + real_team_name[i-1].lower()

    #Create a BeautifulSoup Object
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
        
    #Finding Team Name
    team_tag = soup.find('h1', string=re.compile(' Roster$'))
        
    #Searching For Team Group
    groups = soup.find_all(class_="Table__Title", string=["Offense", "Defense", "Special Teams"])
    #print(len(groups), "groups")
    #for g in groups:
        #print(g)
        
    #Finding the Column Names
    col_names = []
    for g in groups:
        t = g.find_next("table")
        for c in t.thead.tr.children:
            col_names.append(c.text)
        
    #Find the Attributes of a Player
    players = []
    id_list = []
    for x in groups :
        table = x.find_next("table")
        for c in table.tbody.find_all(is_player_row):
            tds = c.find_all('td')
            cols = []
            for td in tds:
                cols.append(td.get_text())
            players.append(cols)
        for aPlayer in players :
            player = {}
            name = mysplit(aPlayer[1], '0123456789')
            player["player_name"] = name[0]

            player_url_name = name[0].split()
            totalURL = ""
            i = 1
            for x in player_url_name :
                if i != len(player_url_name) :
                    if x.find(".") :
                        x = x.replace('.', '') 
                    if x.find('\'') :
                        x = x.replace('\'', '') 
                    totalURL = totalURL + x.lower() + "-"
                else :
                    if x.find(".") :
                        x = x.replace('.', '')
                    if x.find('\'') :
                        x = x.replace('\'', '') 
                    totalURL = totalURL + x.lower()
                i = i + 1
            totalURL = '/' + totalURL
            #print(totalURL)
            #print(soup.find('a', href=re.compile(totalURL))['href'])
            id = soup.find('a', href=re.compile(totalURL))['href']
            id = re.sub('http://www.espn.com/college-football/player/_/id/', '', id)
            id = re.sub(totalURL, '', id)
            if id_list.count(id) == 0: 
                id_list.append(id)
                player["player_id"] = id
                if name[1] == '' :
                    player["player_no"] = "NA"
                else :
                    player["player_no"] = name[1]
                player["POS"] = aPlayer[2]
                player["HT"] = aPlayer[3]
                player["WT"] = aPlayer[4]
                player["Class"] = aPlayer[5]
                player["Birthplace"] = aPlayer[6]
                #print(player)
                roster.append(player)
        
    # YOUR CODE HERE
    #raise NotImplementedError()
    return roster

In [23]:
import pprint
import numpy as np
team_name = "Virginia Cavaliers"
team_id = 258
team_roster = extract_team_roster(team_id, team_name)
pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(team_roster)
player_ids = np.array([p['player_id'] for p in team_roster])
unqiue_ids = np.unique(player_ids)
#assert len(unqiue_ids) == len(team_roster)
#print(len(unqiue_ids))
#print(len(team_roster))
player_nos = np.array([p['player_no'] for p in team_roster])
na_count = len([no for no in player_nos if no == 'NA'])
print(na_count)
#assert na_count > 10

wrong
45


In [24]:
# test the function
import pprint

team_id = 150
team_name = "Duke Blue Devils"
team_roster = extract_team_roster(team_id, team_name)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(team_roster[:3])

wrong
[   {   'Birthplace': 'Grayson, GA',
        'Class': 'JR',
        'HT': '6\' 3"',
        'POS': 'QB',
        'WT': '235 lbs',
        'player_id': '4239994',
        'player_name': 'Chase Brice',
        'player_no': '8'},
    {   'Birthplace': 'Langhorne, PA',
        'Class': 'SO',
        'HT': '6\' 3"',
        'POS': 'QB',
        'WT': '190 lbs',
        'player_id': '4568231',
        'player_name': 'Jack Colyar',
        'player_no': '13'},
    {   'Birthplace': 'Venice, CA',
        'Class': 'FR',
        'HT': '6\' 3"',
        'POS': 'QB',
        'WT': '180 lbs',
        'player_id': '4429578',
        'player_name': 'Luca Diamont',
        'player_no': '2'}]


In [25]:
type(team_roster) is list

True

In [26]:
assert len(team_roster) >= 80

In [27]:
assert ','.join(sorted(team_roster[50].keys())) == 'Birthplace,Class,HT,POS,WT,player_id,player_name,player_no'

In [28]:
import numpy as np
classes = np.array([p['Class'] for p in team_roster])
class_labels, class_counts = np.unique(classes, return_counts=True)
assert ''.join(class_labels.tolist()) == 'FRJRSOSR'

In [29]:
import numpy as np
player_ids = np.array([p['player_id'] for p in team_roster])
unqiue_ids = np.unique(player_ids)
assert len(unqiue_ids) == len(team_roster)

### Question 9. Debug and Testing Functions

When you write a program, you have to make sure that it always works correctly even if some exteral data (e.g., the ESPN website) may be corrupted. For example, Some players in a certain team may not have their player no. Your code should be able tp catch such exceptions and replace the missing player no with the value "NA". (10 points)

In [30]:
# YOUR CODE HERE
#raise NotImplementedError()
import requests
import re
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup
team_name = "Virginia Cavaliers"
team_id = 258

print("wrong3")
roster = []
def mysplit(s, ex):
    head = s.rstrip(ex)
    tail = s[len(head):]
    return head, tail

def is_player_row(tag):
    if tag.name == 'tr' and tag.has_attr('data-idx'):
        return True
    return False

#Computing Team URL
url = "https://www.espn.com/college-football/team/roster/_/id/" + str(team_id) + "/"
real_team_name = team_name.split( )
#print(real_team_name)
i = 1
while (len(real_team_name) > i):     
    url = url + real_team_name[i-1].lower()+"-"
    i = i + 1
else: 
    url = url + real_team_name[i-1].lower()

#Create a BeautifulSoup Object
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, 'html.parser')

#Finding Team Name
team_tag = soup.find('h1', string=re.compile(' Roster$'))

#Searching For Team Group
groups = soup.find_all(class_="Table__Title", string=["Offense", "Defense", "Special Teams"])
#print(len(groups), "groups")
#for g in groups:
    #print(g)

#Finding the Column Names
col_names = []
for g in groups:
    t = g.find_next("table")
    for c in t.thead.tr.children:
        col_names.append(c.text)

#Find the Attributes of a Player
players = []
id_list = []
for x in groups :
    table = x.find_next("table")
    for c in table.tbody.find_all(is_player_row):
        tds = c.find_all('td')
        cols = []
        for td in tds:
            cols.append(td.get_text())
        players.append(cols)
    for aPlayer in players :
        player = {}
        name = mysplit(aPlayer[1], '0123456789')
        player["player_name"] = name[0]

        player_url_name = name[0].split()
        totalURL = ""
        i = 1
        for x in player_url_name :
            if i != len(player_url_name) :
                if x.find(".") :
                    x = x.replace('.', '') 
                if x.find('\'') :
                    x = x.replace('\'', '') 
                totalURL = totalURL + x.lower() + "-"
            else :
                if x.find(".") :
                    x = x.replace('.', '')
                if x.find('\'') :
                    x = x.replace('\'', '') 
                totalURL = totalURL + x.lower()
            i = i + 1
        totalURL = '/' + totalURL
        #print(totalURL)
        #print(soup.find('a', href=re.compile(totalURL))['href'])
        id = soup.find('a', href=re.compile(totalURL))['href']
        id = re.sub('http://www.espn.com/college-football/player/_/id/', '', id)
        id = re.sub(totalURL, '', id)
        if id_list.count(id) == 0: 
            id_list.append(id)
            player["player_id"] = id
            if name[1] == '' :
                player["player_no"] = "NA"
            else :
                player["player_no"] = name[1]
            player["POS"] = aPlayer[2]
            player["HT"] = aPlayer[3]
            player["WT"] = aPlayer[4]
            player["Class"] = aPlayer[5]
            player["Birthplace"] = aPlayer[6]
            #print(player)
            roster.append(player)

team_roster = roster

wrong3


In [31]:
import numpy as np
positions = np.array([p['POS'] for p in team_roster])
position_label, position_counts = np.unique(positions, return_counts=True)
assert ''.join(position_label.tolist()) == 'CCBDBDEDLDTFBGLBLSNTOLOTPPKQBRBSTEWR'

In [32]:
player_nos = np.array([p['player_no'] for p in team_roster])
na_count = len([no for no in player_nos if no == 'NA'])
assert na_count > 10

### Question 10. Add functions into a Python Modules

Now you have developed two solid functions: `get_team_url` and `extract_team_roster` and you would like to reuse them in other projects or share them with other people. Use the Notebook magic command to write the two functions with other neccessary  statements into a python file named `roster.py`. (4 points)

In [33]:
%%writefile roster.py

# YOUR CODE HERE
#raise NotImplementedError()
import requests
import re
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup

def get_team_url(team_id, team_name):
    url = "https://www.espn.com/college-football/team/roster/_/id/" + str(team_id) + "/"
    real_team_name = team_name.split( )
    #print(real_team_name)
    i = 1
    while (len(real_team_name) > i):     
        url = url + real_team_name[i-1].lower()+"-"
        i = i + 1
    else: 
        url = url + real_team_name[i-1].lower()
    return url

def extract_team_roster(team_id, team_name):
    """
    Extract the team roster for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        a list of dict representing the roster of the entire team.
    """
    #team_id = 150
    #team_name = "Duke Blue Devils"
    roster = []
    def mysplit(s, ex):
        head = s.rstrip(ex)
        tail = s[len(head):]
        return head, tail
    
    def is_player_row(tag):
        if tag.name == 'tr' and tag.has_attr('data-idx'):
            return True
        return False
    
    #Computing Team URL
    url = get_team_url(team_id, team_name)

    #Create a BeautifulSoup Object
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
        
    #Finding Team Name
    team_tag = soup.find('h1', string=re.compile(' Roster$'))
        
    #Searching For Team Group
    groups = soup.find_all(class_="Table__Title", string=["Offense", "Defense", "Special Teams"])
    #print(len(groups), "groups")
    #for g in groups:
        #print(g)
        
    #Finding the Column Names
    col_names = []
    for g in groups:
        t = g.find_next("table")
        for c in t.thead.tr.children:
            col_names.append(c.text)
        
    #Find the Attributes of a Player
    players = []
    id_list = []
    for x in groups :
        table = x.find_next("table")
        for c in table.tbody.find_all(is_player_row):
            tds = c.find_all('td')
            cols = []
            for td in tds:
                cols.append(td.get_text())
            players.append(cols)
        for aPlayer in players :
            player = {}
            name = mysplit(aPlayer[1], '0123456789')
            player["player_name"] = name[0]
            
            player_url_name = name[0].split()
            #print(player_url_name)
            totalURL = ""
            i = 1
            for x in player_url_name :
                #print(x)
                if i != len(player_url_name) :
                    if x.find(".") :
                        x = x.replace('.', '') 
                    if x.find('\'') :
                        #print(x)
                        x = x.replace('\'', '') 
                    if x.find(',') :
                        x = x.replace(',', '')
                    #print(x)
                    totalURL = totalURL + x.lower() + "-"
                else :
                    if x.find(".") :
                        x = x.replace('.', '')
                    if x.find('\'') :
                        #print(x)
                        x = x.replace('\'', '') 
                    if x.find(',') :
                        x = x.replace(',', '')
                    if x != '' :
                        #print(x)
                        totalURL = totalURL + x.lower()
                i = i + 1
            totalURL = '/' + totalURL
            #print(totalURL)
            #print(soup.find('a', href=re.compile(totalURL))['href'])
            id = soup.find('a', href=re.compile(totalURL))['href']
            id = re.sub('http://www.espn.com/college-football/player/_/id/', '', id)
            id = re.sub(totalURL, '', id)
            if id_list.count(id) == 0: 
                id_list.append(id)
                player["player_id"] = id
                if name[1] == '' :
                    player["player_no"] = "NA"
                else :
                    player["player_no"] = name[1]
                player["POS"] = aPlayer[2]
                player["HT"] = aPlayer[3]
                player["WT"] = aPlayer[4]
                player["Class"] = aPlayer[5]
                player["Birthplace"] = aPlayer[6]
                #print(player)
                roster.append(player)
        
    # YOUR CODE HERE
    #raise NotImplementedError()
    return roster

Overwriting roster.py


In [34]:
#import os
#import json
#import pandas as pd
#import numpy as np
#from roster import extract_team_roster
#hi = extract_team_roster(2390, 'Miami Hurricanes')
#print(hi)

In [35]:
!cat roster.py


# YOUR CODE HERE
#raise NotImplementedError()
import requests
import re
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup

def get_team_url(team_id, team_name):
    url = "https://www.espn.com/college-football/team/roster/_/id/" + str(team_id) + "/"
    real_team_name = team_name.split( )
    #print(real_team_name)
    i = 1
    while (len(real_team_name) > i):     
        url = url + real_team_name[i-1].lower()+"-"
        i = i + 1
    else: 
        url = url + real_team_name[i-1].lower()
    return url

def extract_team_roster(team_id, team_name):
    """
    Extract the team roster for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        a list of dict representing the roster of the entire team.
    """
    #team_id = 150
    #team_name = "Duke Blue Devils"
    roster = []
    def mysplit(s, ex):
        head = s.rstrip(ex)


In [36]:
import os
assert os.path.exists("roster.py")

In [37]:
try:
    from bs4 import BeautifulSoup
    import requests
    from roster import get_team_url, extract_team_roster
    
    clemson_tiger_roster = extract_team_roster(258, "Virginia Cavaliers")
except:
    raise AssertionError("could not import")


### Congratulations! Yo have finished Part B of the problem.